In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
traffic_data = pd.read_csv('preprocessed_traffic_data.csv')
traffic_data.head()

,day_of_month,hour,dayofweek,latitude,longitude,maximum_speed,minimum_speed,average_speed,number_of_vehicles
0,1,0,2,28.811646,41.080627,135,18,81,132
1,1,0,2,29.108276,40.987244,143,10,73,162
2,1,0,2,29.097290,41.003723,128,6,50,110
3,1,0,2,28.679810,40.998230,111,22,68,101
4,1,0,2,28.020630,41.042175,99,99,99,1


In [13]:
# Select relevant features for training
features = ['day_of_month', 'hour', 'dayofweek', 'latitude', 'longitude', 'maximum_speed', 'minimum_speed', 'average_speed']
X = traffic_data[features].values
y = traffic_data['number_of_vehicles'].values

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape input data for LSTM
sequence_length = 24
X_train_lstm = np.array([X_train[i - sequence_length:i] for i in range(sequence_length, len(X_train))])
y_train_lstm = np.array([y_train[i] for i in range(sequence_length, len(y_train))])

X_test_lstm = np.array([X_test[i - sequence_length:i] for i in range(sequence_length, len(X_test))])
y_test_lstm = np.array([y_test[i] for i in range(sequence_length, len(y_test))])

In [15]:
# Convert data to PyTorch tensors
X_train_lstm_tensor = torch.tensor(X_train_lstm, dtype=torch.float32)
y_train_lstm_tensor = torch.tensor(y_train_lstm, dtype=torch.float32)

X_test_lstm_tensor = torch.tensor(X_test_lstm, dtype=torch.float32)
y_test_lstm_tensor = torch.tensor(y_test_lstm, dtype=torch.float32)

In [16]:
# Define LSTM model
# Due to the time limitation, a simple model could barely be trained. A more complex architecture could be implemented to cath 
# more significant patterns from the data.
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate=0.2):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.dropout(out)
        out, _ = self.lstm2(out)
        out = self.fc(out[:, -1, :])
        return out

In [17]:
# Hyperparameters
input_size = X_train_lstm.shape[2]
hidden_size = 50
num_layers = 1
output_size = 1
learning_rate = 0.001
num_epochs = 5

In [18]:
# Instantiate model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

C:\ProgramData\anaconda3\envs\mytorch\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [19]:
# Split data into batches
batch_size = 64
num_batches = len(X_train_lstm_tensor) // batch_size

losses = []  # to store loss values for plotting

In [10]:
for epoch in range(num_epochs):
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size

        X_batch = X_train_lstm_tensor[start_idx:end_idx]
        y_batch = y_train_lstm_tensor[start_idx:end_idx]

        outputs = model(X_batch)
        optimizer.zero_grad()
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'lstm_traffic_estimation_model.pth')
print('Model saved successfully.')

Epoch [1/5], Loss: 15810.4023
Epoch [2/5], Loss: 15810.3916
Epoch [3/5], Loss: 15810.3926
Epoch [4/5], Loss: 15810.3906
Epoch [5/5], Loss: 15810.3965
Model saved successfully.


In [20]:
# Load the trained weights
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load('lstm_traffic_estimation_model.pth'))

C:\ProgramData\anaconda3\envs\mytorch\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<All keys matched successfully>

In [22]:
# Make predictions on the test set
with torch.no_grad():
    model.eval()
    batch_size = 64  
    predictions = []

    for i in range(0, len(X_test_lstm_tensor), batch_size):
        batch = X_test_lstm_tensor[i:i + batch_size]
        batch_predictions = model(batch).squeeze().numpy()
        predictions.append(batch_predictions)

    predictions = np.concatenate(predictions)

In [23]:
# Evaluate the model
mse = mean_squared_error(y_test_lstm, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_lstm, predictions)
mae = mean_absolute_error(y_test_lstm, predictions)

In [24]:
print(f'Mean Squared Error (MSE) on Test Set: {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) on Test Set: {rmse:.4f}')
print(f'R-squared (R^2) on Test Set: {r2:.4f}')
print(f'Mean Absolute Error (MAE) on Test Set: {mae:.4f}')

Mean Squared Error (MSE) on Test Set: 10912.6161
Root Mean Squared Error (RMSE) on Test Set: 104.4635
R-squared (R^2) on Test Set: -0.0000
Mean Absolute Error (MAE) on Test Set: 70.7456


In [25]:
# For Python version
import sys
print("Python Version:", sys.version)

# For PyTorch version
import torch
print("PyTorch Version:", torch.__version__)

Python Version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch Version: 2.1.2+cpu


In [27]:
# For Jupyter Notebook version
!pip show notebook

Name: notebook
Version: 7.0.6
Summary: Jupyter Notebook - A web-based notebook environment for interactive computing
Home-page: 
Author: 
Author-email: Jupyter Development Team <jupyter@googlegroups.com>
License: BSD 3-Clause License

- Copyright (c) 2001-2015, IPython Development Team
- Copyright (c) 2015-, Jupyter Development Team

All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software